# Azure AI GPT-4 with Vision demo
This demo shows various examples of using GPT-4 with vision.

First, let's set up the environment for connecting to GPT-4 in the Azure Open AI Service by getting the key and endpoint from environment variables and creating an OpenAIClient.

In [23]:
using System;
using System.IO;
using System.Text;
using System.Net.Http;
using System.Threading.Tasks;
using Newtonsoft.Json;

public static string Gpt4Key = System.Environment.GetEnvironmentVariable("Gpt4Key");
public static string Gpt4Endpoint = System.Environment.GetEnvironmentVariable("Gpt4Endpoint");

public static HttpClient httpClient = new System.Net.Http.HttpClient();
httpClient.DefaultRequestHeaders.Add("api-key", Gpt4Key);

Console.WriteLine($"Created client for {Gpt4Endpoint}");

Created client for https://nickward-ai-westus-aiservices-1088222743.openai.azure.com/openai/deployments/gpt-4/chat/completions?api-version=2023-07-01-preview


### Create a re-usable class to prompt GPT-4 with a prompt and an image path

In [24]:
public static async Task<string> PromptGptWithImageAsync(string prompt, string image_path)
{
    string systemMessage = "You are a helpful assistant.";
    float temperature = 0.7f;
    float top_p = 0.95f;
    int max_tokens = 800;
    bool stream = false;

    string encodedImage = Convert.ToBase64String(File.ReadAllBytes(image_path));

    var payload = new
    {
        messages = new object[] {
            new { role = "system", content = new object[] {
                new {
                    type = "text",
                    text = systemMessage
                }
            } },
            new { role = "user", content = new object[] {
                new {
                    type = "image_url",
                    image_url = new {
                        url = $"data:image/jpeg;base64,{encodedImage}"
                    }
                },
                new {
                    type = "text",
                    text = prompt
                }
            } }
        },
        temperature = temperature,
        top_p = top_p,
        max_tokens = max_tokens,
        stream = stream
    };

    var response = await httpClient.PostAsync(Gpt4Endpoint, new StringContent(JsonConvert.SerializeObject(payload), System.Text.Encoding.UTF8, "application/json"));
    if (response.IsSuccessStatusCode)
    {
        var responseData = JsonConvert.DeserializeObject<dynamic>(await response.Content.ReadAsStringAsync());
        return responseData.choices[0].message.content;
    }
    else
    {
        return $"Error: {response.StatusCode}, {response.ReasonPhrase}";
    }
}

## Demo 1 - holiday accommodation rental

Ask GPT4 to look at an image of a holiday rental house and provide a listing title and description for a rental advertisement.

In [26]:
string image_path = "01-log-cabin.jpg";
string userMessage = @"You are an Al assistant which generates listings for vacation rentals. 
Generate exciting content for this image but don't talk about content that you cannot see. 
Follow the format of an attention grabbing title and provide a description that is only 6 sentences long.";

var result = await PromptGptWithImageAsync(userMessage, image_path);
Console.WriteLine(result);


Title: "Cozy Winter Retreat Amidst the Whispers of the Pines"

Nestled in a serene snow-covered landscape, this charming log cabin offers a perfect escape into the heart of nature. As dusk falls, the warm glow from the windows creates an inviting atmosphere, promising a cozy haven from the frosty outdoors. The spacious wrap-around deck, adorned with a blanket of snow, awaits your morning coffee or evening stargazing. Inside, the rustic decor and crackling fireplace set the stage for memorable moments with loved ones. Just steps away from the tranquility of the surrounding forest, this cabin is your gateway to a winter wonderland. Book your stay and indulge in the peaceful solitude that this enchanting woodland abode has to offer.


## Demo 2 - 